In [5]:
# Final Full Dynamic Pricing Notebook with Real-Time Pathway Integration
!pip install pathway bokeh --quiet
# --- Imports and Setup ---
import pathway as pw
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
output_notebook()

# --- Data Simulation Setup with Pathway ---
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float
    LastUpdatedDate: str
    LastUpdatedTime: str
    datetime: str

static_df = pd.read_csv('/content/dataset.csv')
static_df["datetime"] = pd.to_datetime(static_df["LastUpdatedDate"] + " " + static_df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
static_df = static_df.sort_values("datetime")

class DataStreamer(pw.io.python.Connector):
    def __init__(self, df):
        self.df = df
        self.index = 0

    def run(self):
        for _, row in self.df.iterrows():
            yield row.to_dict()

@pw.pipelines.pw_pipeline
def parking_pipeline():
    input_stream = pw.io.python.read(DataStreamer(static_df), schema=ParkingSchema)
    return input_stream

stream = parking_pipeline()
data_stream = static_df.copy()  # Simulate stream manually for real-time like behavior

# --- Model 1: Baseline Linear Model ---
def baseline_model(df, alpha=2.0):
    price_dict = {}
    for lot, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('datetime')
        price = 10.0
        price_series = []
        for _, row in group.iterrows():
            occ_rate = row['Occupancy'] / row['Capacity']
            price += alpha * occ_rate
            price_series.append(price)
        price_dict[lot] = price_series
    return price_dict

# --- Model 2: Custom Demand-Based Model ---
def demand_model(df):
    veh_weight = {'bike': 0.7, 'car': 1.0, 'truck': 1.5}
    traffic_val = {'low': 1, 'medium': 2, 'high': 3}

    demand_vals = []
    for _, row in df.iterrows():
        occ_rate = row['Occupancy'] / row['Capacity']
        queue_rate = row['QueueLength'] / row['Capacity']
        traffic = traffic_val.get(row['TrafficConditionNearby'], 2) / 3.0
        special = row['IsSpecialDay']
        veh = veh_weight.get(row['VehicleType'], 1.0)
        demand = 0.6*occ_rate + 0.3*queue_rate - 0.2*traffic + 0.5*special + veh
        demand_vals.append(demand)
    dem_min, dem_max = min(demand_vals), max(demand_vals)

    price_dict = {}
    i = 0
    for lot, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('datetime')
        price_series = []
        for _, row in group.iterrows():
            occ_rate = row['Occupancy'] / row['Capacity']
            queue_rate = row['QueueLength'] / row['Capacity']
            traffic = traffic_val.get(row['TrafficConditionNearby'], 2) / 3.0
            special = row['IsSpecialDay']
            veh = veh_weight.get(row['VehicleType'], 1.0)
            demand = 0.6*occ_rate + 0.3*queue_rate - 0.2*traffic + 0.5*special + veh
            norm_demand = (demand - dem_min) / (dem_max - dem_min)
            price = 10 * (0.5 + 1.5 * norm_demand)
            price_series.append(price)
            i += 1
        price_dict[lot] = price_series
    return price_dict

# --- Model 3: Competitive Pricing Model ---
def competitive_model(df, alpha=2.0):
    centers = {}
    for lot, group in df.groupby('SystemCodeNumber'):
        centers[lot] = (group['Latitude'].median(), group['Longitude'].median())

    def haversine(lat1, lon1, lat2, lon2):
        from math import radians, sin, cos, sqrt, atan2
        R = 6371.0
        dlat = radians(lat2 - lat1)
        dlon = radians(lon2 - lon1)
        a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c

    neighbors = {}
    for lot1, (lat1, lon1) in centers.items():
        neighs = []
        for lot2, (lat2, lon2) in centers.items():
            if lot1 != lot2 and haversine(lat1, lon1, lat2, lon2) <= 1.0:
                neighs.append(lot2)
        neighbors[lot1] = neighs

    price_dict = {}
    notes = {}
    for lot, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('datetime')
        prev_price = 10.0
        price_series = []
        reroute_series = []
        for _, row in group.iterrows():
            occ_rate = row['Occupancy'] / row['Capacity']
            price = prev_price + alpha * occ_rate

            neighs = neighbors.get(lot, [])
            avg_neigh_price = 10.0
            if neighs:
                prices = []
                for n in neighs:
                    neigh_group = df[df['SystemCodeNumber'] == n]
                    if not neigh_group.empty:
                        occ = neigh_group.iloc[0]['Occupancy'] / neigh_group.iloc[0]['Capacity']
                        prices.append(10.0 + alpha * occ)
                if prices:
                    avg_neigh_price = np.mean(prices)

            note = None
            if occ_rate >= 0.9:
                if avg_neigh_price < price:
                    price *= 0.9
                    note = 'Reroute suggested'
                else:
                    price *= 1.05

            price = max(5, min(price, 20))
            price_series.append(price)
            reroute_series.append(note)
            prev_price = price

        price_dict[lot] = price_series
        notes[lot] = reroute_series
    return price_dict, notes

# --- Visualizations ---
def plot_prices(price_dict, title):
    plots = []
    for lot, prices in price_dict.items():
        p = figure(title=f"{title}: {lot}", x_axis_label='Time Index', y_axis_label='Price')
        p.line(range(len(prices)), prices, legend_label=lot)
        plots.append(p)
    show(column(*plots))

# --- Run All Models ---
model1_prices = baseline_model(data_stream)
model2_prices = demand_model(data_stream)
model3_prices, reroutes = competitive_model(data_stream)

# --- Show Plots ---
plot_prices(model1_prices, "Model 1 - Baseline")
plot_prices(model2_prices, "Model 2 - Demand-Based")
plot_prices(model3_prices, "Model 3 - Competitive")

AttributeError: module 'pathway.io.python' has no attribute 'Connector'